In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, precision_recall_curve,f1_score, fbeta_score, log_loss
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve 
from sklearn.pipeline import Pipeline, make_pipeline
sns.set()


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/metis/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
df_cases = pd.read_csv('covid_cases.csv')
df_deaths = pd.read_csv('covid_deaths.csv')
beds = pd.read_csv('bed_data.csv')
pop = pd.read_csv('covid_county_population.csv')
cvd_dat = pd.read_csv('state_covid_data.csv')

# Population dataframe

In [4]:
pop.drop('State', axis=1, inplace=True)

In [5]:
pop = pop[pop['County Name'] != 'Statewide Unallocated']
df_cases = df_cases[df_cases['County Name'] != 'Statewide Unallocated']
df_deaths = df_deaths[df_deaths['County Name'] != 'Statewide Unallocated']

# Beds dataframe

In [6]:
beds.drop(['State/Local Government', 'Non-Profit', 'For-Profit'], axis=1, inplace=True )

In [7]:
beds['Beds per Person'] = beds.Total / 1000

In [8]:
beds.drop(0, inplace=True)

In [9]:
beds.head()

,Location,Total,Beds per Person
1,Alabama,3.1,0.0031
2,Alaska,2.2,0.0022
3,Arizona,1.9,0.0019
4,Arkansas,3.2,0.0032
5,California,1.8,0.0018


In [10]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [11]:
df_cases.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20
1,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10,10,12,12,12,12,12,12,17,17,19,19,19,23,25,25,25,25,27,28,30,32,33,36
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,3,3,4,4,5,5,10,15,18,19,23,25,28,29,34,38,42,49,59,59,66,71,78,87,98,102,103,109,114,117,123,132,143,147
3,1005,Barbour County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,7,9,10,10,9,11,13,14,15,18,20,22,28,29,30,32
4,1007,Bibb County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,3,4,4,4,7,7,8,9,11,11,13,16,17,17,19,23,23,26,28,32,32,33,33,34
5,1009,Blount County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,5,5,5,5,5,5,6,9,10,10,10,10,10,11,12,12,13,15,16,17,18,20,20,21,22,26,29,31,31


# Covid-19 Features

In [13]:
cases_small = df_cases.iloc[:, :-19]
deaths_small = df_deaths.iloc[:, :-19]

In [14]:
cases_small['Days since case1'] = cases_small.astype(bool).sum(axis=1)
deaths_small['Days since death1'] = deaths_small.astype(bool).sum(axis=1)

In [15]:
cases_small['Days since case1'] = cases_small['Days since case1'] - 4
deaths_small['Days since death1'] = deaths_small['Days since death1'] - 4

In [16]:
cases_small = cases_small[['countyFIPS', 'County Name', 'State', '4/5/20', 'Days since case1']]
deaths_small = deaths_small[['countyFIPS', 'County Name', 'State', '4/5/20', 'Days since death1']]

In [17]:
cases_small.columns = ['countyFIPS', 'County Name', 'State', 'Total_cases', 'Days since case1']
deaths_small.columns = ['countyFIPS', 'County Name', 'State', 'Total_deaths', 'Days since death1']

In [18]:
deaths_small.columns

Index(['countyFIPS', 'County Name', 'State', 'Total_deaths',
       'Days since death1'],
      dtype='object')

In [19]:
cases_small.State = cases_small.State.apply(lambda x: states[x])
deaths_small.State = deaths_small.State.apply(lambda x: states[x])

In [20]:
cases_small = cases_small.merge(beds, left_on='State', right_on='Location')
deaths_small = deaths_small.merge(beds, left_on='State', right_on='Location')

In [21]:
cases_small.drop(['Location', 'Total'], axis=1, inplace=True)
deaths_small.drop(['Location', 'Total'], axis=1, inplace=True)

In [22]:
cases_small = cases_small.merge(pop, left_on='countyFIPS', right_on='countyFIPS')
deaths_small = deaths_small.merge(pop, left_on='countyFIPS', right_on='countyFIPS')

In [23]:
cases_small['County_beds'] = cases_small['Beds per Person'] * cases_small.population
deaths_small['County_beds'] = deaths_small['Beds per Person'] * deaths_small.population

In [24]:
cases_small['County_beds'] = cases_small['County_beds'].apply(int)
deaths_small['County_beds'] = deaths_small['County_beds'].apply(int)

In [25]:
cases_small.drop(['Beds per Person', 'County Name_y'], axis=1, inplace=True )
deaths_small.drop(['Beds per Person', 'County Name_y'], axis=1, inplace=True )

In [26]:
deaths_small.head()

,countyFIPS,County Name_x,State,Total_deaths,Days since death1,population,County_beds
0,1001,Autauga County,Alabama,0,0,55869,173
1,1003,Baldwin County,Alabama,0,0,223234,692
2,1005,Barbour County,Alabama,0,0,24686,76
3,1007,Bibb County,Alabama,0,0,22394,69
4,1009,Blount County,Alabama,0,0,57826,179


In [27]:
deaths_small.drop(['County Name_x', 'State', 'population', 'County_beds'], axis=1, inplace=True)

In [28]:
deaths_small.head()

,countyFIPS,Total_deaths,Days since death1
0,1001,0,0
1,1003,0,0
2,1005,0,0
3,1007,0,0
4,1009,0,0


In [29]:
covid_features = cases_small.merge(deaths_small, left_on='countyFIPS', right_on='countyFIPS')

In [30]:
covid_features.head()

,countyFIPS,County Name_x,State,Total_cases,Days since case1,population,County_beds,Total_deaths,Days since death1
0,1001,Autauga County,Alabama,12,13,55869,173,0,0
1,1003,Baldwin County,Alabama,34,23,223234,692,0,0
2,1005,Barbour County,Alabama,2,3,24686,76,0,0
3,1007,Bibb County,Alabama,7,7,22394,69,0,0
4,1009,Blount County,Alabama,10,12,57826,179,0,0


In [31]:
covid_features.columns

Index(['countyFIPS', 'County Name_x', 'State', 'Total_cases',
       'Days since case1', 'population', 'County_beds', 'Total_deaths',
       'Days since death1'],
      dtype='object')

In [32]:
covid_features.columns = ['countyFIPS', 'County Name', 'State', 'Total_cases',
       'Days since case1', 'population', 'County_beds', 'Total_deaths',
       'Days since death1']

In [33]:
covid_features.head()

,countyFIPS,County Name,State,Total_cases,Days since case1,population,County_beds,Total_deaths,Days since death1
0,1001,Autauga County,Alabama,12,13,55869,173,0,0
1,1003,Baldwin County,Alabama,34,23,223234,692,0,0
2,1005,Barbour County,Alabama,2,3,24686,76,0,0
3,1007,Bibb County,Alabama,7,7,22394,69,0,0
4,1009,Blount County,Alabama,10,12,57826,179,0,0


In [34]:
col_ord = ['countyFIPS', 'County Name', 'State', 'population','County_beds', 'Total_cases', 'Total_deaths',
           'Days since case1', 'Days since death1']

In [35]:
covid_features = covid_features[col_ord]

In [36]:
covid_features.head()

,countyFIPS,County Name,State,population,County_beds,Total_cases,Total_deaths,Days since case1,Days since death1
0,1001,Autauga County,Alabama,55869,173,12,0,13,0
1,1003,Baldwin County,Alabama,223234,692,34,0,23,0
2,1005,Barbour County,Alabama,24686,76,2,0,3,0
3,1007,Bibb County,Alabama,22394,69,7,0,7,0
4,1009,Blount County,Alabama,57826,179,10,0,12,0


## Adding more Covid-19 data

In [37]:
cvd_dat.head()

,Province_State,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
0,Alabama,4888,157,NaN,4731.0,104.246265,45712.0,641.0,3.211948,974.898788,13.113748
1,Alaska,319,9,153.0,310.0,53.366881,9895.0,37.0,2.821317,1655.377090,11.598746
2,American Samoa,0,0,NaN,NaN,0.000000,3.0,NaN,NaN,5.391708,NaN
3,Arizona,4933,184,994.0,4749.0,67.772933,52990.0,631.0,3.729982,728.012918,12.791405
4,Arkansas,1781,39,721.0,1742.0,68.790322,24209.0,291.0,2.189781,935.061705,16.339135


In [38]:
cvd_dat = cvd_dat[['Province_State', 'People_Hospitalized', 'Testing_Rate']]

In [39]:
cvd_dat.head()

,Province_State,People_Hospitalized,Testing_Rate
0,Alabama,641.0,974.898788
1,Alaska,37.0,1655.377090
2,American Samoa,NaN,5.391708
3,Arizona,631.0,728.012918
4,Arkansas,291.0,935.061705


In [40]:
covid_features = covid_features.merge(cvd_dat, left_on='State', right_on='Province_State')

In [41]:
covid_features['County_hospitalizations'] = (covid_features.People_Hospitalized / covid_features.population)*covid_features.Total_cases

    
covid_features['County_testing_rate'] =  covid_features.Testing_Rate / covid_features.population


In [42]:
covid_features.drop(['Province_State', 'Testing_Rate', 'People_Hospitalized'], axis=1, inplace=True)

In [43]:
covid_features = covid_features[covid_features.population != 0]

# Cases and Deaths by Week

In [44]:
weekly_cases = df_cases.iloc[:,4:].diff(axis=1).groupby((np.arange(len(df_cases.columns[4:])) // 7) + 1, axis=1).sum().add_prefix('Week_')
weekly_deaths = df_deaths.iloc[:,4:].diff(axis=1).groupby((np.arange(len(df_deaths.columns[4:])) // 7) + 1, axis=1).sum().add_prefix('Week_')


In [45]:
weekly_cases = weekly_cases.cumsum(axis=1)
weekly_deaths = weekly_deaths.cumsum(axis=1)

In [46]:
weekly_cases['County'] = df_cases['County Name']
weekly_cases['State'] = df_cases['State']
weekly_cases['countyFIPS'] = df_cases['countyFIPS']

weekly_deaths['County'] = df_deaths['County Name']
weekly_deaths['State'] = df_deaths['State']
weekly_deaths['countyFIPS'] = df_deaths['countyFIPS']

In [47]:
col_ord = ['countyFIPS', 'County', 'State', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8',
          'Week_9', 'Week_10', 'Week_11', 'Week_12', 'Week_13', 'Week_14']

In [48]:
weekly_cases = weekly_cases[col_ord]
weekly_deaths = weekly_deaths[col_ord]

In [49]:
weekly_cases.State = weekly_cases.State.apply(lambda x: states[x])

weekly_deaths.State = weekly_deaths.State.apply(lambda x: states[x])

In [50]:
weekly_cases.head()

,countyFIPS,County,State,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14
1,1001,Autauga County,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,12.0,23.0,30.0,36.0
2,1003,Baldwin County,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,19.0,42.0,87.0,123.0,147.0
3,1005,Barbour County,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,28.0,32.0
4,1007,Bibb County,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,8.0,17.0,32.0,34.0
5,1009,Blount County,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,10.0,16.0,26.0,31.0


In [51]:
covid_features.head()

,countyFIPS,County Name,State,population,County_beds,Total_cases,Total_deaths,Days since case1,Days since death1,County_hospitalizations,County_testing_rate
0,1001,Autauga County,Alabama,55869,173,12,0,13,0,0.137679,0.017450
1,1003,Baldwin County,Alabama,223234,692,34,0,23,0,0.097628,0.004367
2,1005,Barbour County,Alabama,24686,76,2,0,3,0,0.051932,0.039492
3,1007,Bibb County,Alabama,22394,69,7,0,7,0,0.200366,0.043534
4,1009,Blount County,Alabama,57826,179,10,0,12,0,0.110850,0.016859


# Weekly Percentage Increases

In [52]:
cases_percent = weekly_cases.copy()
deaths_percent = weekly_deaths.copy()

In [53]:
cases_percent.iloc[: , 4:] = cases_percent.iloc[: , 4:].pct_change(axis=1)
deaths_percent.iloc[: , 4:] = deaths_percent.iloc[: , 4:].pct_change(axis=1)

# Producing Target

In [54]:

def count_values_in_range(series):
    '''Counts entries in a series above 1'''
    return (series > 0.50).sum()

In [55]:
# produces a column containing the numbers of weeks cases in a county increased by 50% or more

cases_percent["Weeks_increased"] = cases_percent.iloc[:, 13:].apply(
    func=lambda row: count_values_in_range(row), axis=1)

In [56]:
cases_percent.Weeks_doubled.value_counts()

1    1145
2     944
0     633
3     385
4      38
Name: Weeks_doubled, dtype: int64

In [57]:
cases_percent['At_risk'] = (cases_percent.Weeks_doubled > 1).astype(int)

In [58]:
cases_percent.At_risk.describe()

count    3145.000000
mean        0.434658
std         0.495791
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: At_risk, dtype: float64

In [59]:
target = cases_percent[['countyFIPS', 'At_risk']]

## The target

In [119]:
target.to_csv('target_table.csv')

# Save Tables

In [61]:
covid_table = covid_features.merge(target, left_on='countyFIPS', right_on='countyFIPS')

In [62]:
covid_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 3141
Data columns (total 12 columns):
countyFIPS                 3142 non-null int64
County Name                3142 non-null object
State                      3142 non-null object
population                 3142 non-null int64
County_beds                3142 non-null int64
Total_cases                3142 non-null int64
Total_deaths               3142 non-null int64
Days since case1           3142 non-null int64
Days since death1          3142 non-null int64
County_hospitalizations    2940 non-null float64
County_testing_rate        3142 non-null float64
At_risk                    3142 non-null int64
dtypes: float64(2), int64(8), object(2)
memory usage: 319.1+ KB


In [290]:
covid_table.fillna(0, inplace=True)

In [291]:
covid_table = covid_table[covid_table.population != 0]

In [68]:
covid_table.to_csv('covid_table.csv')

# on entire set

In [63]:
df = pd.read_csv('proj3_data.csv', index_col=0)

In [64]:
df.drop('at_risk', axis=1, inplace=True)

In [65]:
df = df.merge(target, left_on='countyfips', right_on='countyFIPS')

In [66]:
df.drop('countyFIPS', axis=1, inplace=True)

In [67]:
df.head()

,countyfips,county_name,state,population,county_beds,total_cases,total_deaths,days_since_case1,days_since_death1,county_hospitalizations,county_testing_rate,inc_pov_rate,inc_med_hh,inc_per_cap,jobs_serv,jobs_manu,jobs_agro,jobs_unem,jobs_const,jobs_trans,ppl_ov65_pct,ppl_un18_pct,ppl_nohs_pct,ppl_college_pct,ppl_popdensity,ppl_wht_pct,ppl_hisp_pct,ppl_bl_pct,At_risk
0,1001,Autauga County,Alabama,55869,173,27,1,27,14,0.309778,0.017450,13.4,58343.0,27824.0,41.9086,13.32120,0.605508,3.6,5.13437,7.43613,11.3803,26.78,12.29970,24.9640,91.80,77.25,2.40,17.58,1
1,1003,Baldwin County,Alabama,223234,692,114,1,37,4,0.327343,0.004367,10.1,56607.0,29364.0,45.9202,9.80375,1.028740,3.6,7.60888,4.83541,12.8459,22.99,9.78661,30.7421,114.65,83.50,4.38,9.31,1
2,1005,Barbour County,Alabama,24686,76,20,0,17,0,0.519323,0.039492,33.4,32490.0,17561.0,33.3183,23.75540,4.223920,5.2,5.63190,6.24014,13.8505,21.91,26.90680,12.0321,31.03,46.75,5.05,46.69,1
3,1007,Bibb County,Alabama,22394,69,28,0,21,0,0.801465,0.043534,20.2,45795.0,20911.0,38.9304,20.54830,0.416106,4.0,9.16656,5.94786,10.5697,22.70,17.86370,13.2228,36.81,75.02,1.77,21.92,1
4,1009,Blount County,Alabama,57826,179,21,0,26,0,0.232785,0.016859,12.8,48253.0,22021.0,38.2647,14.81290,1.623010,3.5,10.28530,7.32460,10.2900,24.61,20.21530,13.1020,88.90,88.89,8.07,1.26,1


In [113]:
X, y = df.iloc[:, 3:-1], df.At_risk

X, X_test, y, y_test = train_test_split(X, y, test_size = 0.2)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25)

In [114]:
log_pipe = make_pipeline(StandardScaler(), LogisticRegression(solver= 'liblinear', C=1))

log_pipe.fit(X_train, y_train)

y_test.describe()

count    629.000000
mean       0.429253
std        0.495363
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: At_risk, dtype: float64

In [115]:
log_pipe.score(X_val, y_val)

0.6831210191082803

In [116]:
prediction_hard = log_pipe.predict(X_val)

In [117]:
confusion_matrix(y_val, prediction_hard)

array([[268,  94],
       [105, 161]])

In [118]:
precision_score(y_val, prediction_hard), recall_score(y_val, prediction_hard), roc_auc_score(y_val, prediction_hard), f1_score(y_val, prediction_hard)



(0.6313725490196078, 0.6052631578947368, 0.6727973248037219, 0.618042226487524)